<a href="https://colab.research.google.com/github/nntadotzip/XLNet-sandbox/blob/main/IUOntologyQA_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pd.set_option('display.max_colwidth', None)
conversation = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/preprocessing/IUOntologyQA_withParagraph.csv', encoding='utf8')
conversation.head()

title  ... Unnamed: 5
0  Thông tin chung  ...        NaN
1  Thông tin chung  ...        NaN
2  Thông tin chung  ...        NaN
3  Thông tin chung  ...        NaN
4  Thông tin chung  ...        NaN

[5 rows x 6 columns]

In [ ]:
conversation = conversation.drop(conversation.columns[[4, 5]], axis=1)
conversation.sample(2)

title  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
def remove_separator_whitespace(sentence):
  sentence = sentence.replace('|', ' ')
  sentence = sentence.replace('\t', ' ')
  sentence = sentence.replace('., ', ' ')
  sentence = sentence.replace('..', '.')
  sentence = sentence.replace('. ,', '.')
  sentence = sentence.replace(', .', ',')
  sentence = ' '.join(sentence.split())
  return sentence

In [ ]:
conversation['context'] = conversation['context'].map(remove_separator_whitespace)
conversation['question'] = conversation['question'].map(remove_separator_whitespace)
conversation['answer'] = conversation['answer'].map(remove_separator_whitespace)
conversation.head(1)

title  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
import regex as re

def handle_word_begin_with_dot(sentence):
  #regex pattern of word begin with dot. i.e., .diemChuan
  word_begin_with_dot = '\.[^\s\d]\w+'
  list_of_dot_words = re.findall(word_begin_with_dot, sentence)
  dot_word_split = [x.replace('.', '. ') for x in list_of_dot_words]
  correction_dict = {list_of_dot_words[i]: dot_word_split[i] for i in range(len(list_of_dot_words))}
  for incorrect, correct in correction_dict.items():
    sentence = sentence.replace(incorrect, correct)
  return sentence

In [ ]:
conversation.context = conversation.context.map(handle_word_begin_with_dot)
conversation.answer = conversation.answer.map(handle_word_begin_with_dot)
conversation.head(1)

title  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
def find_answer_startPosition(context, answer):
  return context.find(answer)

def find_answer_endPosition(context, answer):
  return (context.find(answer) + len(answer))

In [ ]:
start_position = conversation.apply(lambda x: find_answer_startPosition(x.context, x.answer), axis=1)
end_position = conversation.apply(lambda x: find_answer_endPosition(x.context, x.answer), axis=1)

In [ ]:
conversation = conversation.assign(start_position = start_position)
conversation = conversation.assign(end_position = end_position)
conversation.sample(1)

title  ... end_position
56  Thông tin chung  ...         3535

[1 rows x 6 columns]

In [ ]:
conversation[conversation.start_position ==-1]

,title,question,answer,context,start_position,end_position


In [ ]:
conversation.to_csv('conversation_formatted.csv',encoding='utf-8-sig',index=False)

In [ ]:
conversation.context.nunique()

6

# Expected Format:

{
  'version': 'version_no',
  'data': [
    {},
    {},
    ...
    {
      'title': 'title',
      'paragraph': [
        {
          'qas': [
            {
              'question': '',
              'id': '',
              'answers': [
                {
                  'text': '',
                  'answer_start': ''
                }
              ]
            }
          ],
          'context': ''
        },
      ]
    }
  ]
}

```
# This is formatted as code
{
  'version': 'version_no',
  'data': [
    {},
    {},
    ...
    {
      'title': 'title',
      'paragraph': [
        {
          'qas': [
            {
              'question': '',
              'id': '',
              'answers': [
                {
                  'text': '',
                  'answer_start': ''
                }
              ]
            }
          ],

          'context': ''
        },
      ]
    }
  ]
}

```



In [ ]:
from json import encoder
from numpy import add
from collections import defaultdict
import json

In [ ]:
data_path = '/content/conversation_formatted.csv'
output_path ='output.json'

_version = 'v1.0'
_dict = defaultdict(list)

In [ ]:
def create_qas(qas, question, id, answer, is_impossible):
    qas.append({'question': question, 'id': id, 'answers': answer, 'is_impossible': is_impossible})
    return qas

def create_paragraph(paragraph, qas, context):
    paragraph.append({'qas': qas, 'context': context})
    return paragraph

def create_answers(text, answer_start):
    return [{'text': text, 'answer_start': answer_start}]

def create_data(data, title, paragraph):
    data.append({'title': title, 'paragraphs': paragraph})
    return data

In [ ]:
df = pd.read_csv(data_path)
df

title  ... end_position
0             Thông tin chung  ...          646
1             Thông tin chung  ...         1409
2             Thông tin chung  ...         2071
3             Thông tin chung  ...         2633
4             Thông tin chung  ...         3283
..                        ...  ...          ...
180  Phương thức tuyển sinh  ...          272
181  Phương thức tuyển sinh  ...          410
182  Phương thức tuyển sinh  ...          474
183  Phương thức tuyển sinh  ...          573
184  Phương thức tuyển sinh  ...          665

[185 rows x 6 columns]

In [ ]:
_dict = {'version': _version}
context = None
title = ''
paragraph = []
data = []

for index, value in df.iterrows():
    answer = create_answers(value['answer'], value['start_position'])
    is_impossible = False
    if context != value['context']:
        if index != 0:
            paragraph = create_paragraph(paragraph, qas, context)

        context = value['context']
        qas = []

    qas = create_qas(qas, value['question'], index, answer, is_impossible)

    # check wheather the title change
    if title != value['title']:
        if index != 0:
            data = create_data(data, title, paragraph)

        title = value['title']
        paragraph = []

    if index == df.shape[0]-1:
        paragraph = create_paragraph(paragraph,qas,context)
        data = create_data(data, title, paragraph)
        break

_dict['data'] = data
json_object = json.dumps(_dict, indent=4, ensure_ascii=False)

with open(output_path, "w", encoding='utf8') as outfile:
    outfile.write(json_object)

# Train Test Split

In [ ]:
import json
from sklearn.model_selection import train_test_split

In [ ]:
f = open('output.json',)
data = json.load(f)['data']

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=2021)

In [ ]:
with open('train.json', 'w', encoding='utf-8') as f:
    json.dump(train, f, ensure_ascii=False, indent=4)

In [ ]:
with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(test, f, ensure_ascii=False, indent=4)